In [59]:
import openai
from openai import AsyncOpenAI
from dotenv import load_dotenv
import os
import asyncio
from rich import print as rprint
load_dotenv()

True

In [7]:
# Initialize OpenAI client
client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [32]:
openai_messages = [
    {
        "role": "user",
        "content": "Replace the last paragraph with a short story about Tiptap",
    }
]
system_message = "You are an assistant that can edit rich text documents."

In [45]:
async def stream_chat_completion():
    stream = await client.responses.create(
        model="gpt-5-mini",
        instructions=system_message,
        input=openai_messages,
        # tools=get_tool_definitions(),
        stream=True,
    )
    async for chunk in stream:
            # Handle different chunk types from Responses API
            if hasattr(chunk, "output") and chunk.output:
                for output_item in chunk.output:
                    if hasattr(output_item, "content"):
                        for content_item in output_item.content:
                            if hasattr(content_item, "text") and content_item.text:
                                yield {"type": "content", "content": content_item.text}

                    # Handle tool calls
                    if hasattr(output_item, "input") and hasattr(output_item, "name"):
                        yield {
                            "type": "tool_call",
                            "tool_call": {
                                "id": f"call_{hash(str(output_item.input))}",
                                "name": output_item.name,
                                "arguments": str(output_item.input),
                            },
                        }

In [46]:
async def stream_and_collect():
    chunks = []
    async for chunk in stream_chat_completion():
        chunks.append(chunk)
        print(chunk, end="")  # Real-time display
    return chunks

In [47]:
all_chunks = await stream_and_collect()

In [48]:
all_chunks

[]

In [61]:
from openai import OpenAI
import json

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "Get today's horoscope for an astrological sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "sign": {
                    "type": "string",
                    "description": "An astrological sign like Taurus or Aquarius",
                },
            },
            "required": ["sign"],
        },
    },
]

def get_horoscope(sign):
    return f"{sign}: Next Tuesday you will befriend a baby otter."

# Create a running input list we will add to over time
input_list = [
    {"role": "user", "content": "What is my horoscope? I am an Aquarius."}
]

# 2. Prompt the model with tools defined
response = client.responses.create(
    model="gpt-5",
    tools=tools,
    input=input_list,
)

# Save function call outputs for subsequent requests
input_list += response.output
print(response.output)

for item in response.output:
    if item.type == "function_call":
        if item.name == "get_horoscope":
            # 3. Execute the function logic for get_horoscope
            horoscope = get_horoscope(json.loads(item.arguments))
            
            # 4. Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "horoscope": horoscope
                })
            })


print("Final input:")
print(input_list)

response = client.responses.create(
    model="gpt-5",
    instructions="Respond only with a horoscope generated by a tool.",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

[ResponseReasoningItem(id='rs_68c1bd4cea4c8199be6adae71ae3a1eb0b85d00ed8cebbe8', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"sign":"Aquarius"}', call_id='call_gQBHd7mvf5o7rBCEv0Nyr29h', name='get_horoscope', type='function_call', id='fc_68c1bd4ecdf481999be3113c8accd0f50b85d00ed8cebbe8', status='completed')]
Final input:
[{'role': 'user', 'content': 'What is my horoscope? I am an Aquarius.'}, ResponseReasoningItem(id='rs_68c1bd4cea4c8199be6adae71ae3a1eb0b85d00ed8cebbe8', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"sign":"Aquarius"}', call_id='call_gQBHd7mvf5o7rBCEv0Nyr29h', name='get_horoscope', type='function_call', id='fc_68c1bd4ecdf481999be3113c8accd0f50b85d00ed8cebbe8', status='completed'), {'type': 'function_call_output', 'call_id': 'call_gQBHd7mvf5o7rBCEv0Nyr29h', 'output': '{"horoscope": "{\'sign\': \'Aquarius\'}: Next Tuesday you

In [76]:
input_list = [
    {
        "role": "user",
        "content": "What is my horoscope? I am an Aquarius."
    },
    # Removed invalid fields: status, encrypted_content, summary
    {
        "id": "rs_68c1bd4cea4c8199be6adae71ae3a1eb0b85d00ed8cebbe8",
        "type": "reasoning",
        "summary": [],
        "content": None,
    },
    # Removed invalid field: status
    {
        "arguments": "{\"sign\":\"Aquarius\"}",
        "call_id": "call_gQBHd7mvf5o7rBCEv0Nyr29h",
        "name": "get_horoscope",
        "type": "function_call",
        "id": "fc_68c1bd4ecdf481999be3113c8accd0f50b85d00ed8cebbe8",
    },
    {
        "type": "function_call_output",
        "call_id": "call_gQBHd7mvf5o7rBCEv0Nyr29h",
        "output": "{\"horoscope\": \"{'sign': 'Aquarius'}: Next Tuesday you will befriend a baby otter.\"}"
    }
]

In [77]:
response = client.responses.create(
    model="gpt-5",
    instructions="Respond only with a horoscope generated by a tool.",
    tools=tools,
    input=input_list,
)

In [79]:
print(response.model_dump_json(indent=2))

{
  "id": "resp_68c1c0396a388199b28f9148848105d80b85d00ed8cebbe8",
  "created_at": 1757528122.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "Respond only with a horoscope generated by a tool.",
  "metadata": {},
  "model": "gpt-5-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "rs_68c1c03bb2a081998bdde2dc2b9a5add0b85d00ed8cebbe8",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "msg_68c1c03e0c508199ab734884f462ba390b85d00ed8cebbe8",
      "content": [
        {
          "annotations": [],
          "text": "{'sign': 'Aquarius'}: Next Tuesday you will befriend a baby otter.",
          "type": "output_text",
          "logprobs": []
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "

In [67]:
input_list_2= [
    {
        "role": "user",
        "content": "Replace the last paragraph with a short story about Tiptap",
    },
    {
        "id": "rs_68c1a3fc9fcc819a896b3abe9ff6892c00eecbc3a9769c42",
        "type": "reasoning",
        "content": None,
        "summary": [],
    },
    {
        "type": "function_call",
        "id": "fc_68c1a3fc9fcc819a896b3abe9ff6892c00eecbc3a9769c42",
        "call_id": "call_qW4TBOy9d2ulcR5N7WslhfND",
        "name": "readFirstChunk",
        "arguments": "{}",
        "status": "completed",
    },
    {
        "type": "function_call_output",
        "call_id": "call_qW4TBOy9d2ulcR5N7WslhfND",
        "output": 'Amount of chunks: 1\nCurrent chunk index: 0\nContent:\n"""\n<h1>AI Agent Demo</h1><p>Ask the AI to improve this.</p>\n"""\n',
    },
]

In [68]:
from ai_integration import get_tool_definitions

In [69]:
response = client.responses.create(
    model="gpt-5",
    instructions="Respond only with a horoscope generated by a tool.",
    tools=get_tool_definitions(),
    input=input_list_2,
)

NotFoundError: Error code: 404 - {'error': {'message': "Item with id 'rs_68c1a3fc9fcc819a896b3abe9ff6892c00eecbc3a9769c42' not found.", 'type': 'invalid_request_error', 'param': 'input', 'code': None}}

In [70]:
NotFoundError: Error code: 404 - {'error': {'message': "Item with id 'rs_68c1a3fc9fcc819a896b3abe9ff6892c00eecbc3a9769c42' not found.", 'type': 'invalid_request_error', 'param': 'input', 'code': None}}

SyntaxError: invalid syntax (3347176368.py, line 1)